In [99]:
import nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, leaguegamefinder, playerdashboardbyclutch, playergamelogs, commonplayerinfo, teamplayeronoffdetails
import pandas as pd

In [225]:
class Player(object):
    def __init__(self, PlayerID, TeamID):
    
        self.PlayerID =  PlayerID 
        print(self.PlayerID, type(self.PlayerID))
        self.PlayerName = players.find_player_by_id(player_id=self.PlayerID)['full_name']
        print(self.PlayerName)
        self.TeamID = TeamID

        self.StartYear = commonplayerinfo.CommonPlayerInfo(player_id= self.PlayerID).get_data_frames()[0]['FROM_YEAR'][0] 
        if self.StartYear < 2010:
            self.StartYear = 2010
            

        self.LastYear = commonplayerinfo.CommonPlayerInfo(player_id= self.PlayerID).get_data_frames()[0]['TO_YEAR'][0] 
        #self.Data = self.compile_active_player_data()

    def compile_active_player_data(self):
        print(222)
        game_date_raw_data = []
        win_loss_raw_data = []
        free_throws_raw_data = [] 
        feild_goals_2pts_raw_data = []
        feild_goals_3pts_raw_data = []
        matchup_raw_data = []
        
        
        for year in range((self.StartYear), (self.LastYear)):
            print("-------------------------------------------------")
            season = str(year) + "-" + str(((year+1)-2000))
            playerLogs = playergamelogs.PlayerGameLogs(player_id_nullable=self.PlayerID, season_nullable=season)
           
            print(season)
            game_date_api_data = playerLogs.get_data_frames()[0]["GAME_DATE"]
            win_loss_api_data = playerLogs.get_data_frames()[0]["WL"]
            free_throws_api_data = playerLogs.get_data_frames()[0]["FTM"]
            feild_goals_2pts_api_data = playerLogs.get_data_frames()[0]["FGM"]
            feild_goals_3pts_api_data = playerLogs.get_data_frames()[0]["FG3M"]
            matchup_api_data = playerLogs.get_data_frames()[0]["MATCHUP"]
            
            
            
            for game in range(0, len(game_date_api_data)):
                    game_date_raw_data.append(game_date_api_data[game])
                    win_loss_raw_data.append(win_loss_api_data[game])
                    free_throws_raw_data.append(free_throws_api_data[game]) 
                    feild_goals_2pts_raw_data.append(feild_goals_2pts_api_data[game])
                    feild_goals_3pts_raw_data.append(feild_goals_3pts_api_data[game])
                    matchup_raw_data.append(matchup_api_data[game])
        
        player_data = {
        "Game Date": game_date_raw_data,
        "Win/Loss": win_loss_raw_data,
        "Free Throws": free_throws_raw_data,
        "Feild Goals 2pts": feild_goals_2pts_raw_data,
        "Feild Goals 3pts": feild_goals_3pts_raw_data,
        "Matchup": matchup_raw_data
        }

        player_dataFrame = pd.DataFrame(data=player_data)
        print(4444)
        return(player_dataFrame)


    def get_active_player_data(self):
        print(1111)
        print("Why is this not printing")
        return(self.compile_active_player_data())

        #def get_active_player_data(self):


    



In [229]:
class Team(object): 
    def __init__(self,TeamName):
        self.TeamName = TeamName
        self.TeamID = teams.find_teams_by_full_name("Portland Trail Blazers")[0]['id']
        #self.ActivePlayerLists = compile_active_player_DataFrame()

    def compile_active_player_DataFrame(self):
        
        active_team_player_raw =teamplayeronoffdetails.TeamPlayerOnOffDetails(team_id=self.TeamID).get_data_frames()[1]

        name_list = []
        id_list = []
        for player in range(0, len(active_team_player_raw["VS_PLAYER_NAME"])):
            
           
            name_list.append(active_team_player_raw["VS_PLAYER_NAME"][player])
            id_list.append(active_team_player_raw["VS_PLAYER_ID"][player])
            
        
        ActivePlayer_List = {
            "Name": name_list,
            "Player ID": id_list
        }
       
        ActivePlayer_DataFrame = pd.DataFrame(data=ActivePlayer_List)
        
        return(ActivePlayer_DataFrame)
        

    def compile_active_team_player_data(self):
        
        player_DataFrame = self.compile_active_player_DataFrame()
        
        player_data = []
        for i in range(0, len(player_DataFrame["Name"])):
            player_object = player_DataFrame["Name"][i]
            player_id= player_DataFrame["Player ID"][i]
            print("iteration", i)
            print(player_id, type(player_id))
            try:
                player_object = Player(player_id, self.TeamID) #TODO: Make Player ID auto
                print("hahahaha ")
                player_data += player_object.get_active_player_data()
            except:
                print("Error for", player_DataFrame["Name"][i])
            

        player_data = {
            "Player Data":player_data
        }
        player_data_DataFrame = pd.DataFrame(data=player_data)
        return(player_data_DataFrame)
        #player_data_DataFrame["Name"] = player_DataFrame[]
            
   
    



In [233]:
test = Team("Portland Trail Blazers")
print(test.compile_active_team_player_data()[0])



ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)